# DTOcean Environmental Impact (Hydro Stage) Test

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [4]:
import numpy as np

In [5]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [6]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [7]:
# Bring up the logger
start_logging()

2016-12-07 16:00:31,740 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [8]:
new_core = Core()
project_menu = ProjectMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [9]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2016-12-07 16:00:37,125 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2016-12-07 16:00:37,127 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2016-12-07 16:00:37,128 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [10]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

2016-12-07 16:00:37,155 - INFO - aneris.control.data - New "device.system_type" data stored with index JFRARE
2016-12-07 16:00:37,157 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [11]:
project_menu.initiate_pipeline(new_core, new_project)

2016-12-07 16:00:37,176 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2016-12-07 16:00:37,187 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index 5DDSA6
2016-12-07 16:00:37,188 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2016-12-07 16:00:37,190 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2016-12-07 16:00:37,193 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [12]:
names_themes = theme_menu.get_available(new_core, new_project)
message = html_list(names_themes)
HTML(message)


## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [13]:
theme_name = 'Environmental Impact Assessment'
theme_menu.activate(new_core, new_project, theme_name)


## Check the status of the module inputs

In [14]:
environmental_branch = pipe_tree.get_branch(new_core, new_project, 'Environmental Impact Assessment')
input_status = environmental_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [ ]:
project_menu.initiate_dataflow(new_core, new_project)

2016-12-07 16:00:37,265 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index KLJIN7
2016-12-07 16:00:37,266 - INFO - aneris.control.simulation - Datastate stored
2016-12-07 16:00:37,269 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [ ]:
%run inputs_environmental.py

In [ ]:
environmental_branch.read_test_data(new_core,
                            new_project,
                            "inputs_environmental.pkl")

## Check if the module can be executed

In [ ]:
can_execute = theme_menu.is_executable(new_core, new_project, theme_name)
display(can_execute)

In [ ]:
input_status = environmental_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [ ]:
theme_menu.execute_all(new_core, new_project)

## Examine the results

Currently, there is no robustness built into the core, so the assumption is that the module executed successfully. This will have to be improved towards deployment of the final software.

Let's check the number of devices and annual output of the farm, using just information in the data object.

In [ ]:
output_status = environmental_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [ ]:
global_eis= new_core.get_data_value(new_project, "farm.global_eis")

global_eis


In [ ]:
import math
math.isnan(hydro_global_eis["Negative Impact"])

## Plotting some graphs

By having data objects with set formats it should be possible to create automated plot generation. However, some plots may be too complex and some special cases may need defined.

In [ ]:
annual_energy_per_dev_value = new_core.get_data_value(new_project, 
                                                     "farm.annual_energy_per_device")
meta = new_core.get_metadata("farm.annual_energy_per_device")

chart_values = np.array(annual_energy_per_dev_value.values())

In [ ]:
plt.bar(range(len(annual_energy_per_dev_value)),
        chart_values,
        align='center')
            
plt.xticks(range(len(annual_energy_per_dev_value)),
           annual_energy_per_dev_value.keys())
plt.title(meta.title)

plt.ylabel(meta.units[0])

plt.tight_layout()
# plt.savefig('annual_energy_per_device.png')
plt.show()

## Plotting the Layout

This may require such a special case. It is not clear is a new data type is required or just special plots associated to variable IDs.

In [ ]:
layout_value = new_core.get_data_value(new_project, "farm.layout")
layout_meta = new_core.get_metadata("farm.layout")

In [ ]:
x = []
y = []

for coords in layout_value.itervalues():
    
    x.append(coords.x)
    y.append(coords.y)

fig = plt.figure()
ax1 = fig.add_subplot(1,1,1, axisbg='lightskyblue')
ax1.plot(x,y,'k+', mew=2, markersize=10)
plt.title(layout_meta.title)
plt.axis('equal')
plt.show()